#### we know that predictiong pos based on the previous pos is best. now we modify the method od fill OOV words. instead of fill by the most frequent pos, we find the most similar word to this missing one and then fill those word pos

In [1]:
import utils
from transformers import RobertaTokenizerFast, RobertaForMaskedLM
import torch
from collections import Counter
import random

In [2]:
train_file = './data/pos/ass1-tagger-train'
train_sentences = utils.get_file_data(train_file)
train_words_pos = utils.get_all_words_pos(train_sentences)
train_data_freq = utils.get_word_freq_per_pos(train_words_pos)

Reading file


In [3]:
dev_input_file = './data/pos/ass1-tagger-dev-input'
dev_file = './data/pos/ass1-tagger-dev'
dev_sentences = utils.get_file_data(dev_file)
dev_actual = [pos for _, pos in utils.get_all_words_pos(dev_sentences)]
dev_input_sentences = utils.get_file_data(dev_input_file)

Reading file
Reading file


In [4]:
train_pos_word_freq = utils.get_bigram_dict(train_sentences)
train_pos_word_freq

{'_In': {'IN': 1664, 'RB': 1, 'RBR': 1},
 'IN_an': {'DT': 1173},
 'DT_Oct.': {'NNP': 21},
 'NNP_19': {'CD': 26},
 'CD_review': {'NN': 1},
 'NN_of': {'IN': 13650},
 'IN_``': {'``': 564},
 '``_The': {'DT': 474, 'NNP': 19},
 'DT_Misanthrope': {'NN': 3},
 "NN_''": {"''": 664},
 "''_at": {'IN': 21},
 'IN_Chicago': {'NNP': 75},
 "NNP_'s": {'POS': 5015, 'VBZ': 35, 'NNP': 1, 'NNS': 1},
 'POS_Goodman': {'NNP': 1},
 'NNP_Theatre': {'NNP': 4},
 'NNP_(': {'(': 302},
 '(_``': {'``': 41},
 '``_Revitalized': {'VBN': 1},
 'VBN_Classics': {'NNS': 1},
 'NNS_Take': {'VBP': 1},
 'VBP_the': {'DT': 705, 'JJ': 1},
 'DT_Stage': {'NN': 1},
 'NN_in': {'IN': 5229, 'RP': 4, 'RB': 2},
 'IN_Windy': {'NNP': 1},
 'NNP_City': {'NNP': 116},
 'NNP_,': {',': 12686},
 ",_''": {"''": 2758},
 "''_Leisure": {'NN': 1},
 'NN_&': {'CC': 3},
 'CC_Arts': {'NNS': 2, 'NNPS': 1},
 'NNS_)': {')': 61},
 ')_,': {',': 225},
 ',_the': {'DT': 3383},
 'DT_role': {'NN': 25},
 'IN_Celimene': {'NNP': 3},
 ',_played': {'VBN': 1},
 'VBN_by': {'

In [15]:
def get_top_k_known_predictions2(logits, word_index, tokenizer, known_vocab, k=5):
    '''without ensuring known words'''
    top_k_values, top_k_indices = torch.topk(logits[0, word_index, :], k)
    top_k_probabilities = torch.nn.functional.softmax(top_k_values, dim=-1)[:k]
    predicted_tokens = []
    for i in top_k_indices:
        decoded_i = tokenizer.decode(i)
        if (decoded_i in known_vocab) and (decoded_i.isalpha()) and (len(decoded_i)>1):
            predicted_tokens += [decoded_i]
    return {'predicted_tokens': predicted_tokens, 'probas': top_k_probabilities}

def get_top_k_known_predictions(logits, word_index, tokenizer, known_vocab, k=5):
    '''ensuring known words'''
    probabilities = torch.nn.functional.softmax(logits[0, word_index, :], dim=-1)
    sort_indices = torch.argsort(probabilities, descending =True)
    predicted_tokens = []
    predicted_probabilities = []
    for i in sort_indices:
        decoded_i = tokenizer.decode(i)
        if (decoded_i in known_vocab) and (decoded_i.isalpha()) and (len(decoded_i)>1):
            predicted_tokens += [decoded_i]
            predicted_probabilities += [probabilities[i].item()]

    top_k_predictions = predicted_tokens[:k]
    top_k_probabilities = predicted_probabilities[:k]
    
    return {'predicted_tokens': top_k_predictions, 'probas': top_k_probabilities}

In [18]:
import time

def bigram_predict_with_contextualized(train_pos_word_dist_data, train_pos_dist_data, test_data, fill_model_name = 'roberta-base', k=5):
    start = time.time()
    cum_fill_time = 0
    random.seed(42)
    word_pos_pred = []
    train_pos_word_most_freq_pos = utils.get_word_most_freq_pos(train_pos_word_dist_data)
    train_word_most_freq_pos = utils.get_word_most_freq_pos(train_pos_dist_data)
    fill_most_freq_pos = utils.fill_for_missing_word(train_pos_dist_data)
    tokenizer = RobertaTokenizerFast.from_pretrained(fill_model_name)
    masked_language_model = RobertaForMaskedLM.from_pretrained(fill_model_name)
    known_vocab = set(train_pos_dist_data.keys())
        
    for sentence in test_data:
        prev_pos = ''
        for i, word in enumerate(sentence.split()):
            if f'{prev_pos}_{word}' not in train_pos_word_dist_data:
                if word not in train_pos_dist_data:
                    # try first to get similar words before filling with mode
                    sentence_list = sentence.split()
                    sentence_list[i] = tokenizer.mask_token
                    masked_sentence = ' '.join(sentence_list)
                    tokenized_sentence = tokenizer(masked_sentence, return_tensors='pt')
                    mask_index = tokenized_sentence["input_ids"][0].tolist().index(tokenizer.mask_token_id)
                    with torch.no_grad():
                        logits = masked_language_model(**tokenized_sentence).logits
                    start_fill = time.time()
                    top_k_words = get_top_k_known_predictions2(logits, mask_index, tokenizer, known_vocab, k)['predicted_tokens']
                    cum_fill_time += (time.time()-start_fill)
                    # print(cum_fill_time)
                    possible_pos_fills = [train_pos_word_most_freq_pos.get(f'{prev_pos}_{w}', train_word_most_freq_pos.get(w, fill_most_freq_pos)) for w in top_k_words]
                    possible_pos_fills = [fill_most_freq_pos] if possible_pos_fills==[] else possible_pos_fills
                    fill_value = max(possible_pos_fills[::-1], key=possible_pos_fills.count)
                    pos = fill_value
                else:
                    pos = train_word_most_freq_pos[word]
            else:
                pos = train_pos_word_most_freq_pos[f'{prev_pos}_{word}']

            word_pos_pred.append(pos)
            prev_pos = pos

    print(time.time()-start, cum_fill_time, time.time()-start-cum_fill_time, )
    return word_pos_pred


In [19]:
# for k_param in [1,2,3,4,5,6,7]:
dev_predict_bigrm = bigram_predict_with_contextualized(train_pos_word_freq, train_data_freq, dev_input_sentences, k=5)
print(5, 'bigram prediction: ', utils.calc_Accuracy(dev_predict_bigrm, dev_actual))

88.55469274520874 0.568223237991333 87.98646950721741
5 bigram prediction:  0.9291888391474505


In [ ]:
dev_words = set([word for word, _ in utils.get_all_words_pos(dev_sentences)])
train_oov_pos_result = {}
train_pos_count = Counter([p for _, p in train_words_pos])
train_pos_count_overlap = Counter([p for w, p in train_words_pos if w in dev_words])
for p in train_pos_count:
    train_oov_pos_result[p] = train_pos_count_overlap[p]/train_pos_count[p]

sorted(train_oov_pos_result.items(), key=lambda x: x[1])

[('FW', 0.2222222222222222),
 ('SYM', 0.22413793103448276),
 ('UH', 0.27835051546391754),
 ('NNPS', 0.47399925177702956),
 ('NNP', 0.5678394157391817),
 ('VBG', 0.6001616597063182),
 ('JJ', 0.6949703513729846),
 ('NNS', 0.6990777866880513),
 ('VBN', 0.7091490211745904),
 ('LS', 0.7222222222222222),
 ('CD', 0.7388427040035004),
 ('NN', 0.7575732500846278),
 ('VB', 0.7886375671382102),
 ('JJR', 0.8437306979617047),
 ('VBZ', 0.8594961240310077),
 ('RB', 0.877623506619309),
 ('VBD', 0.8913981732409917),
 ('JJS', 0.8967642526964561),
 ('VBP', 0.8988871987831238),
 ('RBR', 0.9688914027149321),
 ('WRB', 0.9780681287914139),
 ('$', 0.9833152468800869),
 (':', 0.9838642078792958),
 ('WP', 0.9843419382141346),
 ('PRP', 0.9908809359944941),
 ('MD', 0.9915332041211874),
 ('PDT', 0.9918478260869565),
 ('PRP$', 0.9920304508147972),
 ('RBS', 0.9933481152993349),
 ('CC', 0.9936558287073751),
 ('WDT', 0.9946436888681882),
 ('IN', 0.9947989097179729),
 ('RP', 0.9981217129977461),
 ('TO', 0.9993869262012